In [1]:
import sqlite3
import pandas as pd
import numpy as np
import time
from utils import *
from ReactNode import ReactNode
from ChemNode import ChemNode
from MCTS import MCTS
from tree_utils import *

In [2]:
conn = sqlite3.connect('buyable.db')
buyable = conn.cursor()
retrobiocat = pd.read_pickle("data/final_test_retrobiocat.pkl")
analyzer = Retrosim()

In [3]:
smile = "C#C[C@]1([C@H](C[C@@H](O1)N2C=NC3=C(N=C(N=C32)F)N)O)CO"
root = ChemNode(smile, 0, None, buyable, retrobiocat, analyzer, None)

In [4]:
mcts = MCTS(root)

In [5]:
start = time.time()
mcts.MCTS(1000)
print("Time taken (s): ", time.time()-start)


100%|██████████| 1000/1000 [13:06<00:00,  1.27it/s]

Time taken (s):  786.2977592945099


In [6]:
for react in root.reactions:
    print(f'Reaction name: {react.reaction_name}, Score: {react.get_mcts_value()}')
    for precursor in react.precursors:
        print(f'Chem {precursor.smiles}, Score: {precursor.score}, visits: {precursor.visits}, solution: {precursor.solution}')
    print(" ")

Reaction name: Phosphate hydrolysis, Score: 1.1398120060417474
Chem C#C[C@]1(COP(=O)(O)O)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 1.9, visits: 14, solution: False
 
Reaction name: Ester hydrolysis (OH), Score: 1.1557994935507891
Chem C#C[C@]1(COC(=O)CC)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O, Score: 0.855, visits: 12, solution: False
 
Reaction name: PNP, Score: 1.16684361707945
Chem C#C[C@]1(CO)O[C@@H](OP(=O)([O-])O)C[C@@H]1O, Score: -0.171475, visits: 504, solution: False
Chem Nc1nc(F)nc2[nH]cnc12, Score: 1.0, visits: 1, solution: True
 
Reaction name: Phosphate hydrolysis, Score: 1.1398120060417474
Chem C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1OP(=O)(O)O, Score: 1.9, visits: 14, solution: False
 
Reaction name: PNP (Incorporated from RDEnzyme), Score: 1.1667038674016121
Chem C#C[C@]1(CO)O[C@H](OP(=O)([O-])[O-])C[C@@H]1O, Score: 5.051625, visits: 566, solution: True
Chem Nc1nc(F)nc2[nH]cnc12, Score: 1.0, visits: 1, solution: True
 
Reaction name: Ketone reduction, Score: 1.

In [7]:
root.solution

True

In [8]:
# Prune the tree starting from the root
prune_tree(root)
paths = generate_paths(root)
print(len(paths))

3


In [12]:
path_explorer(paths[1])

BRANCH 0:
-----------------
Chem: C#C[C@]1(CO)O[C@@H](n2cnc3c(N)nc(F)nc32)C[C@@H]1O
Reaction: PNP (Incorporated from RDEnzyme)
SPLIT: Chem1: C#C[C@]1(CO)O[C@H](OP(=O)([O-])[O-])C[C@@H]1O, Chem2: Nc1nc(F)nc2[nH]cnc12

Reaction: PPM (Incorporated from RDEnzyme)
Chem: C#C[C@]1(COP(=O)([O-])[O-])OC(O)C[C@@H]1O
Reaction: Hemiacetal closure (Incorporated from RDEnzyme)
SPLIT: Chem1: C#C[C@](O)(C=O)COP(=O)([O-])[O-], Chem2: CC=O

Reaction: Phosphorylation
Chem: C#C[C@](O)(C=O)CO
Reaction: Oxidoreductase (Incorporated from RDEnzyme)
Chem: C#CC(O)(CO)CO
BUYABLE

BRANCH 1:
-----------------
Chem: CC=O
BUYABLE

BRANCH 2:
-----------------
Chem: Nc1nc(F)nc2[nH]cnc12
BUYABLE

